In [1]:
import pandas as pd

In [2]:
file_path_episodes = "../exploration/BIG_DF_ML.ipynb"

db01 = pd.read_csv("../gitignore/title_basics_traite.csv")
db02 = pd.read_csv("../gitignore/title_ratings_final.tsv", sep="\t")
db03 = pd.read_csv("../gitignore/title.akas_final.tsv", sep="\t")
db04 = pd.read_csv("../gitignore/tmdb_ml_final.csv")
db05 = pd.read_csv("../gitignore/data_bechdel.csv")
db07 = pd.read_csv("../gitignore/name.basics.tsv", sep="\t") 
db08 = pd.read_csv("../gitignore/title.crew.tsv", sep="\t") 

In [3]:
db07[db07['nconst'] == 'nm0000318']

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
317,nm0000318,Tim Burton,1958,\N,"producer,writer,miscellaneous","tt0099487,tt1142977,tt0121164,tt0408236"


In [4]:
db08[db08['tconst'] == 'tt0107688']

,tconst,directors,writers
106682,tt0107688,nm0783139,"nm0000318,nm0568313,nm0003031"


In [5]:
dbmerge_1 = pd.merge(db01, db02, right_on='title_ratings_tconst', left_on='tconst', how='left') #Title Basics + Title Ratings
dbmerge_2 = pd.merge(dbmerge_1, db03, left_on='tconst', right_on='titleId', how='left') # + Title Akas
dbmerge_3 = pd.merge(dbmerge_2, db04, left_on='tconst', right_on='tmdb_imdb_id', how='left') # + TMDB Full
dbmerge_4 = pd.merge(dbmerge_3, db05, left_on='tconst', right_on='imdbid', how='left') # + Bechdel
dbmerge_4 = pd.merge(dbmerge_4, db08, left_on='tconst', right_on='tconst', how='left') # + Title Crew
dbmerge_4 = pd.merge(dbmerge_4, db07, left_on='directors', right_on='nconst', how='left') # + Name Basics

In [6]:
dbmerge_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688341 entries, 0 to 688340
Columns: 117 entries, tconst to knownForTitles
dtypes: bool(28), float64(10), int64(3), object(76)
memory usage: 485.8+ MB


In [7]:
#dbmerge_4.to_csv("../gitignore/BIG_DF_ML.csv", index=False)

In [8]:
dbmerge_4['movie'].value_counts()

movie
True    688341
Name: count, dtype: int64

In [9]:
BIG_DF_ML = dbmerge_4.drop(columns=[
    'titleType',
    'genres', 
    'decade', 
    'Adult',
    'Short',
    'movie',
    'tmdb_Comedy',
    'tmdb_Adventure',
    'tmdb_Drama',
    'tmdb_Crime',
    'tmdb_Action',
    'tmdb_Documentary',
    'tmdb_Animation',
    'tmdb_Mystery',
    'tmdb_Horror',
    'tmdb_Western',
    'tmdb_Science Fiction',
    'tmdb_Thriller',
    'tmdb_Romance',
    'tmdb_Fantasy',
    'tmdb_Family',
    'tmdb_History',
    'tmdb_Music',
    'tmdb_War', 
    'ordering',
    'region',
    'language',
    'types',
    'attributes',
    'isOriginalTitle',
    'birthYear',
    'deathYear',
    'primaryProfession',
    'knownForTitles',
    'directors',
    'writers'
])

In [10]:
BIG_DF_ML.loc[(dbmerge_4['title_ratings_averageRating'].isnull())&(dbmerge_4['tmdb_vote_average'].isnull()), ['title_ratings_tconst', 'title_ratings_averageRating', 'tmdb_vote_average']]

,title_ratings_tconst,title_ratings_averageRating,tmdb_vote_average
9,NaN,NaN,NaN
10,NaN,NaN,NaN
11,NaN,NaN,NaN
12,NaN,NaN,NaN
13,NaN,NaN,NaN
...,...,...,...
688329,NaN,NaN,NaN
688335,NaN,NaN,NaN
688336,NaN,NaN,NaN
688337,NaN,NaN,NaN


In [11]:
BIG_DF_ML3 = BIG_DF_ML.dropna(subset=['title_ratings_averageRating','tmdb_vote_average'],how='all')

In [12]:
def moyenne_ponderee(ligne):

    # Si 'title_ratings_averageRating' est NaN, on ne prend que 'tmdb_vote_average', et vice versa

    if pd.isna(ligne['title_ratings_averageRating']) and not pd.isna(ligne['tmdb_vote_average']):
        return ligne['tmdb_vote_average']  # Si title_ratings_averageRating est vide, prendre tmdb_vote_average

    elif pd.isna(ligne['tmdb_vote_average']) and not pd.isna(ligne['title_ratings_averageRating']):
        return ligne['title_ratings_averageRating']  # Si tmdb_vote_average est vide, prendre title_ratings_averageRating

    elif not pd.isna(ligne['title_ratings_averageRating']) and not pd.isna(ligne['tmdb_vote_average']):
        # Si les deux colonnes ont des valeurs, calculer la moyenne pondérée
        return (ligne['title_ratings_averageRating'] * ligne['title_ratings_numVotes'] + ligne['tmdb_vote_average'] * ligne['tmdb_vote_count']) / (ligne['title_ratings_numVotes'] + ligne['tmdb_vote_count'])  # Moyenne simple, à ajuster si besoin !
    else:
        return np.nan  # Si les deux sont NaN, retourner NaN


BIG_DF_ML3['notes'] = BIG_DF_ML3.apply(moyenne_ponderee, axis=1)

C:\Users\frale\AppData\Local\Temp\ipykernel_17420\2696331554.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BIG_DF_ML3['notes'] = BIG_DF_ML3.apply(moyenne_ponderee, axis=1)


In [13]:
BIG_DF_ML4 = BIG_DF_ML3.drop(['title_ratings_averageRating','tmdb_vote_average','title_ratings_tconst','titleId','tmdb_imdb_id','imdbid', 'primaryName'], axis=1)

In [14]:
def startyear(ligne):

    # Si 'start year' est 0, il prend la valeur 'tmbd release date'
    BIG_DF_ML4.loc[(BIG_DF_ML4['startYear'] == 0)&(BIG_DF_ML4['tmdb_release_date'] != 0), 'startYear'] = BIG_DF_ML4['tmdb_release_date']

startyear(BIG_DF_ML4)

In [15]:
def release_date(ligne):

    # Si 'tmdn release date' est 0, il prend la valeur 'start year'
    BIG_DF_ML4.loc[(BIG_DF_ML4['tmdb_release_date'] == 0)&(BIG_DF_ML4['startYear'] != 0), 'tmdb_release_date'] = BIG_DF_ML4['startYear']

release_date(BIG_DF_ML4)

In [16]:
def release_date(ligne):

    # Si 'tmdn release date' est 0, il prend la valeur 'start year'
    BIG_DF_ML4.loc[(BIG_DF_ML4['tmdb_release_date'].isna())&(BIG_DF_ML4['startYear'] != 0), 'tmdb_release_date'] = BIG_DF_ML4['startYear']

release_date(BIG_DF_ML4)

In [17]:
def runtimeMinutes(ligne):

    # Si 'runtimeMinutes' est 0, il prend la valeur 'tmdb_runtime'
    BIG_DF_ML4.loc[(BIG_DF_ML4['runtimeMinutes'] == 0)&(BIG_DF_ML4['tmdb_runtime'] != 0), 'runtimeMinutes'] = BIG_DF_ML4['tmdb_runtime']

runtimeMinutes(BIG_DF_ML4)

In [18]:
def runtimeMinutes(ligne):

    # Si 'tmdb_runtime' est 0, il prend la valeur 'runtimeMinutes'
    BIG_DF_ML4.loc[(BIG_DF_ML4['tmdb_runtime'] == 0)&(BIG_DF_ML4['runtimeMinutes'] != 0), 'tmdb_runtime'] = BIG_DF_ML4['runtimeMinutes']

runtimeMinutes(BIG_DF_ML4)

In [19]:
#On transforme les valeurs 0 en NaN pour les supprimer ensuite
import numpy as np
BIG_DF_ML4[BIG_DF_ML4['startYear']==0] = np.nan

C:\Users\frale\AppData\Local\Temp\ipykernel_17420\2379037837.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  BIG_DF_ML4[BIG_DF_ML4['startYear']==0] = np.nan
C:\Users\frale\AppData\Local\Temp\ipykernel_17420\2379037837.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  BIG_DF_ML4[BIG_DF_ML4['startYear']==0] = np.nan
C:\Users\frale\AppData\Local\Temp\ipykernel_17420\2379037837.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  BIG_DF_ML4[BIG_DF_ML4['startYear']==0

In [20]:
#On transforme les valeurs 0 en NaN pour les supprimer ensuite
BIG_DF_ML4[BIG_DF_ML4['runtimeMinutes']==0] = np.nan

In [21]:
BIG_DF_ML5 = BIG_DF_ML4.drop(columns =['tmdb_runtime','tmdb_release_date','tmdb_original_title','tmdb_title','tmdb_vote_count','tmdb_TV Movie'])

In [22]:
BIG_DF_ML5 = BIG_DF_ML5.dropna(subset=['runtimeMinutes'])

In [23]:
BIG_DF_ML5 = BIG_DF_ML5.dropna(subset=['startYear'])

In [24]:
BIG_DF_ML5 = BIG_DF_ML5.dropna(subset=['title'])

In [25]:
# Pour notre ML, on remplace toutes les valeurs nulles des pays de production par False 
Remplacer = [
    'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
    'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE', 'tmdb_SU',
    'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR', 'tmdb_RU', 'tmdb_DK',
    'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC',
    'tmdb_FI', 'tmdb_NO', 'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'
]
BIG_DF_ML5[Remplacer] = BIG_DF_ML5[Remplacer].fillna(False)

C:\Users\frale\AppData\Local\Temp\ipykernel_17420\61637864.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  BIG_DF_ML5[Remplacer] = BIG_DF_ML5[Remplacer].fillna(False)


In [26]:
BIG_DF_ML5['rating'] = BIG_DF_ML5['rating'].fillna(0)

In [27]:
BIG_DF_ML5['title_ratings_numVotes'] = BIG_DF_ML5['title_ratings_numVotes'].fillna(0)

In [28]:
BIG_DF_ML5['tmdb_popularity'] = BIG_DF_ML5['tmdb_popularity'].fillna(0)

In [29]:
BIG_DF_ML5['nconst'] = BIG_DF_ML5['nconst'].fillna("")

In [30]:
BIG_DF_ML5[['Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']] = BIG_DF_ML5[['Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']].astype(bool)

In [31]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

import seaborn as sns
import matplotlib.pyplot as plt

In [32]:
export = "../machine learning/DF_ML.csv.gz"
BIG_DF_ML5.to_csv(export, sep=",", index=False, compression="gzip")

In [33]:
df_ml = BIG_DF_ML5

# Test de ML avec cible sur les notes > 7

In [34]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Index: 301086 entries, 0 to 688339
Data columns (total 69 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   tconst                  301086 non-null  object 
 1   startYear               301086 non-null  float64
 2   runtimeMinutes          301086 non-null  float64
 3   Action                  301086 non-null  bool   
 4   Adventure               301086 non-null  bool   
 5   Animation               301086 non-null  bool   
 6   Biography               301086 non-null  bool   
 7   Comedy                  301086 non-null  bool   
 8   Crime                   301086 non-null  bool   
 9   Documentary             301086 non-null  bool   
 10  Drama                   301086 non-null  bool   
 11  Family                  301086 non-null  bool   
 12  Fantasy                 301086 non-null  bool   
 13  Game-Show               301086 non-null  bool   
 14  History                 3

In [35]:
# Obtenir les fréquences des valeurs uniques dans la colonne 'nconst'
counts = df_ml['nconst'].value_counts()

# Filtrer les valeurs qui apparaissent plus de 20 fois
valid_nconst = counts[counts > 50].index

# Garder uniquement les lignes où 'nconst' est dans valid_nconst
filtered_df = df_ml[df_ml['nconst'].isin(valid_nconst)]

# Résultat
print(filtered_df)

           tconst  startYear  runtimeMinutes  Action  Adventure  Animation  \
8       tt0000679     1908.0           120.0   False       True      False   
20      tt0000941     1909.0            45.0   False      False      False   
50      tt0001184     1910.0            58.0   False       True      False   
82      tt0001614     1911.0            60.0   False      False      False   
113     tt0002031     1912.0            33.0   False      False      False   
...           ...        ...             ...     ...        ...        ...   
688116  tt9904250     2019.0            63.0   False      False      False   
688125  tt9904648     2022.0            82.0   False      False      False   
688144  tt9905476     2019.0            90.0   False      False      False   
688288  tt9914368     2019.0           115.0    True      False      False   
688303  tt9914972     2021.0            96.0   False      False      False   

        Biography  Comedy  Crime  Documentary  ...  tmdb_FI  tm

In [36]:
df_ml2 = df_ml[df_ml['startYear']>1920]
df_ml2['nconst'].nunique()

103218

On veut entrainer notre modèle sur tout le dataframe et afficher UNIQUEMENT les k films les plus proches dont les notes sont supérieures à 7.

In [37]:
def recommandation(tconst):

     df_ml = pd.read_csv("../machine learning/DF_ML.csv.gz")

     index = df_ml.index
     df_ml_num = df_ml.select_dtypes('number')
     df_ml_cat = df_ml.select_dtypes(['object', 'category', 'string', 'bool'])

     from sklearn.preprocessing import MinMaxScaler
     SN = MinMaxScaler()
     df_ml_num_SN = pd.DataFrame(SN.fit_transform(df_ml_num), columns=df_ml_num.columns, index=index)

     df_ml_encoded = pd.concat([df_ml_num_SN, df_ml_cat], axis=1)

     #On crée une liste des colonnes à utiliser pour le modèle
     caracteristiques = df_ml_encoded.columns.drop(['tconst', 'nconst', 'title', 'tmdb_popularity', 'title_ratings_numVotes','tmdb_US',
          'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
          'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
          'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
          'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
          'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
          'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'])

     #On sépare notre df en deux groupes, en fonction de la note
     bons_films = df_ml_encoded[df_ml_encoded['notes'] >= 0.7]
     mauvais_films = df_ml_encoded[df_ml_encoded['notes'] < 0.7]

     #On crée notre modèle
     model = NearestNeighbors(n_neighbors=1000, metric='euclidean')
     model.fit(bons_films[caracteristiques])

     #On déclare les caractéristiques du film sélectionné par l'utilisateur
     caract_film = df_ml_encoded[df_ml_encoded['tconst'] == tconst]
     caract_film = caract_film[caracteristiques]
     caract_film

     distances, indices = model.kneighbors(caract_film)

     #On affiche la selection des films en fonction des indices trouvés par le modèle
     if caract_film['notes'].values[0] > 0.7:
          distances = distances[0][1:11]
          indices = indices[0][1:11]
          selection = bons_films.iloc[indices]['tconst']
     else:
          distances = distances[0][0:10]
          indices = indices[0][0:10]
          selection = bons_films.iloc[indices]['tconst']

     return selection


In [38]:
selection = recommandation('tt0099487')
pd.DataFrame(selection)

,tconst
56355,tt0099653
125111,tt0421715
169677,tt1368491
189246,tt1655441
267928,tt5580390
16838,tt0038348
16369,tt0037671
46018,tt0081534
19208,tt0041719
66256,tt0119643


In [39]:
"""
weights = X_encoded[['Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']].astype(bool)
weights *= 2
X_weighted = pd.concat([weights, X_encoded.drop(columns = ['Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western'])], axis=1)
"""

"\nweights = X_encoded[['Action', 'Adventure',\n       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',\n       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',\n       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',\n       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']].astype(bool)\nweights *= 2\nX_weighted = pd.concat([weights, X_encoded.drop(columns = ['Action', 'Adventure',\n       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',\n       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',\n       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',\n       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western'])], axis=1)\n"

# Test ML avec TF_IDF (sans les poids)

In [40]:
tmdb = pd.read_csv("../gitignore/tmdb_full.csv")

C:\Users\frale\AppData\Local\Temp\ipykernel_17420\3263337905.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  tmdb = pd.read_csv("../gitignore/tmdb_full.csv")


In [41]:
tmdb = tmdb[['imdb_id', 'genres', 'overview']]

In [42]:
df_test = pd.merge(left=df_ml, right=tmdb, how='left', left_on='tconst', right_on='imdb_id')

In [43]:
def recommandation2(tconst):

     index = df_test.index
     df_test_num = df_test.select_dtypes('number')
     df_test_cat = df_test.select_dtypes(['object', 'category', 'string', 'bool'])

     from sklearn.preprocessing import MinMaxScaler
     SN = MinMaxScaler()
     df_test_num_SN = pd.DataFrame(SN.fit_transform(df_test_num), columns=df_test_num.columns, index=index)

     df_test_encoded = pd.concat([df_test_num_SN, df_test_cat], axis=1)

     #On crée une liste des colonnes à utiliser pour le modèle
     caracteristiques = df_test_encoded.columns.drop(['tconst', 'nconst', 'title', 'tmdb_popularity', 
                                                      'title_ratings_numVotes', 'imdb_id', 'genres', 'overview', 'tmdb_US',
          'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
          'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
          'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
          'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
          'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
          'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'])

     #On sépare notre df en deux groupes, en fonction de la note
     bons_films = df_test_encoded[df_test_encoded['notes'] >= 0.7]
     mauvais_films = df_test_encoded[df_test_encoded['notes'] < 0.7]

     #On crée notre modèle
     model = NearestNeighbors(n_neighbors=1000, metric='euclidean')
     model.fit(bons_films[caracteristiques])

     #On déclare les caractéristiques du film sélectionné par l'utilisateur
     caract_film = df_test_encoded[df_test_encoded['tconst'] == tconst]
     caract_film = caract_film[caracteristiques]
     caract_film

     distances, indices = model.kneighbors(caract_film)

     #On affiche la selection des films en fonction des indices trouvés par le modèle
     colonnes = ['tconst', 'genres', 'overview', 'nconst']  # Liste à étendre si besoin
     selection = bons_films.iloc[indices[0]][colonnes]
     df_selection = pd.DataFrame(selection).reset_index()

     #On crée un nouveau df à partir des données textuelles pour l'étudier avec TF-IDF
     df_tfidf = df_selection.copy()
     df_tfidf['texte'] = df_selection[colonnes].fillna('').agg(' '.join, axis=1)
     df_tfidf = pd.DataFrame(df_tfidf['texte'], columns=['texte'])

     from sklearn.feature_extraction.text import TfidfVectorizer
     from sklearn.metrics.pairwise import cosine_similarity

     #Limite aux 100 mots les plus présents et supprime les mots contenus dans plus de 80% des lignes
     vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8, max_features=100)
     df_tfidf = vectorizer.fit_transform(df_tfidf['texte'])

     #On calcule la distance entre chaque vecteur
     similarite = cosine_similarity(df_tfidf)

     #On crée une liste de tuple avec un index et le score de similarite par rapport au film cible (index 0 car première ligne)
     film_index = df_selection.index[0]
     similarity_scores = list(enumerate(similarite[film_index]))

     #On trie le résultat par rapport aux scores et par ordre decroissant en ignorant la partie index (key permet de cibler uniquement le score)
     similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

     similar_movies = [df_selection['tconst'].iloc[i[0]] for i in similarity_scores[1:10]]

     return similar_movies


     

In [44]:
selection2 = recommandation2('tt0099487')
selection2

['tt0120334',
 'tt2905772',
 'tt0041373',
 'tt0054005',
 'tt0345425',
 'tt0241303',
 'tt0213847',
 'tt0098384',
 'tt0029608']

# Test ML avec TF_IDF (avec les poids)

In [45]:
def recommandation2(tconst):

     index = df_test.index
     df_test_num = df_test.select_dtypes('number')
     df_test_cat = df_test.select_dtypes(['object', 'category', 'string', 'bool'])

     from sklearn.preprocessing import MinMaxScaler
     SN = MinMaxScaler()
     df_test_num_SN = pd.DataFrame(SN.fit_transform(df_test_num), columns=df_test_num.columns, index=index)

     df_test_encoded = pd.concat([df_test_num_SN, df_test_cat], axis=1)

     #On crée une liste des colonnes à utiliser pour le modèle
     caracteristiques = df_test_encoded.columns.drop(['tconst', 'nconst', 'title', 'tmdb_popularity', 
                                                      'title_ratings_numVotes', 'imdb_id', 'genres', 'overview', 'tmdb_US',
          'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
          'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
          'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
          'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
          'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
          'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'])

     #On sépare notre df en deux groupes, en fonction de la note
     bons_films = df_test_encoded[df_test_encoded['notes'] >= 0.7]
     mauvais_films = df_test_encoded[df_test_encoded['notes'] < 0.7]

     #On crée notre modèle
     model = NearestNeighbors(n_neighbors=1000, metric='euclidean')
     model.fit(bons_films[caracteristiques])

     #On déclare les caractéristiques du film sélectionné par l'utilisateur
     caract_film = df_test_encoded[df_test_encoded['tconst'] == tconst]
     caract_film = caract_film[caracteristiques]
     caract_film

     distances, indices = model.kneighbors(caract_film)

     #On affiche la selection des films en fonction des indices trouvés par le modèle
     colonnes = ['tconst', 'genres', 'overview', 'nconst']  # Liste à étendre si besoin
     selection = bons_films.iloc[indices[0]][colonnes]
     df_selection = pd.DataFrame(selection).reset_index()

     #On crée une colonne pour identifier le réalisateur s'il correspond à celui du film cible
     cible_real = df_test.loc[df_test['tconst'] == tconst, 'nconst']
     df_selection['real_identique'] = df_selection['nconst'].apply(
        lambda x: 1 if x in cible_real else 0
     )

     #On définit les poids pour chaque colonne textuelle
     colonnes_poids = {
        'tconst': 1,
        'genres': 1,
        'overview': 1,
        'nconst': 20,
        'real_identique': 100
     }

     #On crée une colonne 'texte' pondérée dynamiquement
     df_tfidf = df_selection.copy()
     df_tfidf['texte'] = df_selection.apply(
     lambda ligne: ' '.join(
          (str(ligne[col]) + ' ') * poids for col, poids in colonnes_poids.items()
     ),
     axis=1
     )

     #On crée le DataFrame final pour TF-IDF
     df_tfidf = pd.DataFrame(df_tfidf['texte'], columns=['texte'])

     from sklearn.feature_extraction.text import TfidfVectorizer
     from sklearn.metrics.pairwise import cosine_similarity

     #Limite aux 100 mots les plus présents et supprime les mots contenus dans plus de 80% des lignes
     vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
     df_tfidf = vectorizer.fit_transform(df_tfidf['texte'])

     #On calcule la distance entre chaque vecteur
     similarite = cosine_similarity(df_tfidf)

     #On crée une liste de tuple avec un index et le score de similarite par rapport au film cible (index 0 car première ligne)
     film_index = df_selection.index[0]
     similarity_scores = list(enumerate(similarite[film_index]))

     #On trie le résultat par rapport aux scores et par ordre decroissant en ignorant la partie index (key permet de cibler uniquement le score)
     similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

     similar_movies = [df_selection['tconst'].iloc[i[0]] for i in similarity_scores[1:10]]

     return similarity_scores

     
     

##### Faire un test avec lemmarizing (garder les mots en 'binômes').

In [46]:
def recommandation2(tconst):
    index = df_test.index
    df_test_num = df_test.select_dtypes('number')
    df_test_cat = df_test.select_dtypes(['object', 'category', 'string', 'bool'])

    # Normalize numerical columns
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    df_test_num_SN = pd.DataFrame(SN.fit_transform(df_test_num), columns=df_test_num.columns, index=index)

    # Encode categorical columns
    from sklearn.preprocessing import LabelEncoder
    df_test_cat_encoded = df_test_cat.copy()
    
    for col in df_test_cat_encoded.columns:
        le = LabelEncoder()
        df_test_cat_encoded[col] = le.fit_transform(df_test_cat_encoded[col].fillna('unknown'))
        

    # Combine numeric and encoded data
    df_test_encoded = pd.concat([df_test_num_SN, df_test_cat_encoded], axis=1)
    df_test_encoded['tconst'] = df_test['tconst']  # Add 'tconst' back if it was dropped

    # List of features
    caracteristiques = df_test_encoded.columns.drop(
        ['tconst', 'title', 'tmdb_popularity', 'title_ratings_numVotes', 'imdb_id', 'genres', 'overview', 'tmdb_US',
          'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
          'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
          'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
          'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
          'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
          'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA']
    )

    # Split into "good" and "bad" films
    bons_films = df_test_encoded[df_test_encoded['notes'] >= 0.7]

    # Check if tconst exists
    if tconst not in df_test_encoded['tconst'].values:
        raise ValueError(f"tconst {tconst} not found in the dataset.")

    # Select features for the requested film
    caract_film = df_test_encoded[df_test_encoded['tconst'] == tconst][caracteristiques]

    # NearestNeighbors model
    from sklearn.neighbors import NearestNeighbors
    model = NearestNeighbors(n_neighbors=1000, metric='euclidean')
    model.fit(bons_films[caracteristiques])

    distances, indices = model.kneighbors(caract_film)

    # Select films found
    colonnes = ['tconst', 'genres', 'overview', 'nconst']
    selection = bons_films.iloc[indices[0]][colonnes]
    df_selection = pd.DataFrame(selection).reset_index()

    # Director weighting
    target_directors = df_test.loc[df_test['tconst'] == tconst, 'nconst'].unique()
    df_selection['director_match'] = df_selection['nconst'].apply(
        lambda x: 1 if x in target_directors else 0
    )

    # Weighted column for text
    colonnes_poids = {
        'tconst': 1,
        'genres': 1,
        'overview': 15,
        'nconst': 1
    }

    df_tfidf = df_selection.copy()
    df_tfidf['texte'] = df_selection.apply(
        lambda ligne: ' '.join(
            (str(ligne[col]) + ' ') * poids for col, poids in colonnes_poids.items()
        ),
        axis=1
    )

    # TF-IDF and cosine similarity
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
    df_tfidf_matrix = vectorizer.fit_transform(df_tfidf['texte'])

    similarite = cosine_similarity(df_tfidf_matrix)
    similarity_scores = list(enumerate(similarite[0]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    similar_movies = [df_selection['tconst'].iloc[i[0]] for i in similarity_scores[1:10]]

    return similar_movies

In [47]:
selection2 = recommandation2('tt1375666')
selection2

['tt0482571',
 'tt15398776',
 'tt0154506',
 'tt0278504',
 'tt0372784',
 'tt5013056',
 'tt0816692',
 'tt6723592',
 'tt0468569']

#Pour la prochaine fois : utiliser df_selection pour le TFIDF et ne pas avoir les 300K lignes.

In [48]:


['tt15398776', 'tt0154506', 'tt0482571', 'tt0278504', 'tt0372784', 'tt5013056', 'tt0816692', 'tt6723592', 'tt0468569']

['tt15398776',
 'tt0154506',
 'tt0482571',
 'tt0278504',
 'tt0372784',
 'tt5013056',
 'tt0816692',
 'tt6723592',
 'tt0468569']

In [ ]:
import nltk
nltk.download('popular', quiet=True)
nltk.download('punkt_tab', quiet=True)
from nltk.corpus import stopwords
import spacy
!python -m spacy download en_core_web_lg --quiet
nlp = spacy.load('en_core_web_lg')


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


AttributeError: 'float' object has no attribute 'lower'

In [283]:
'''from tqdm import tqdm
tqdm.pandas()

# On crée une fonction lemmatizing sur le df global pour ne pas avoir à le faire à chaque fois sur le Streamlit
def lemma (texte):
        texte_tokens = nlp(str(texte).lower())

        lemma = []

        for mot in texte_tokens:
            if str(mot).isalpha() == True:
                lemma.append(mot.lemma_)

        sw = stopwords.words('english')
   
        tokens_propres = [mot for mot in lemma if mot not in sw]
        texte_lem = ""
        for mot in tokens_propres:
            texte_lem = texte_lem + " " + mot
            
        return texte_lem

df_test['overview_lem'] = df_test['overview'].progress_apply(lemma)'''

100%|██████████| 301086/301086 [26:54<00:00, 186.44it/s] 


In [ ]:
'''df_test = df_test.drop(columns=['tmdb_US','tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
 'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
 'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
 'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
 'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'])'''

KeyError: "['tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT', 'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE', 'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR', 'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'] not found in axis"

In [285]:
'''export = "../machine learning/DF_ML.csv.gz"
df_test.to_csv(export, sep=",", index=False, compression="gzip")'''

In [286]:
df_test = "../machine learning/DF_ML.csv.gz"

In [247]:
def recommandation6(tconst):
    
    # ----------------------------------------------------------
    # Préparation des données
    # ----------------------------------------------------------

    index = df_test.index
    df_test_num = df_test.select_dtypes('number')
    df_test_cat = df_test.select_dtypes(['object', 'category', 'string', 'bool'])

    # Normalisation des colonnes numériques
    SN = MinMaxScaler()
    df_test_num_SN = pd.DataFrame(SN.fit_transform(df_test_num), columns=df_test_num.columns, index=index)

    # Encodage uniquement de la colonne 'nconst'
    df_test_cat_encoded = df_test_cat.copy()
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df_test_cat_encoded['nconst'] = le.fit_transform(df_test_cat_encoded['nconst'].fillna('inconnu'))

    # On assemble le df numérique et le df texte
    df_test_encoded = pd.concat([df_test_num_SN, df_test_cat_encoded], axis=1)

    #On sépare notre df en deux groupes, en fonction de la note
    bons_films = df_test_encoded[df_test_encoded['notes'] >= 0.7]

    # ----------------------------------------------------------
    # KNN sur les caractéristiques numériques
    # ----------------------------------------------------------

    colonnes_a_exclure = ['tconst', 'title', 'tmdb_popularity', 'title_ratings_numVotes', 'imdb_id', 'genres', 'overview', 'overview_lem']
    caracteristiques = df_test_encoded.columns.drop(colonnes_a_exclure, errors='ignore')

    model = NearestNeighbors(n_neighbors=1000, metric='euclidean') # Il faudra tester d'autres combinaisons
    model.fit(bons_films[caracteristiques])

    caract_film = df_test_encoded[df_test_encoded['tconst'] == tconst][caracteristiques]
    distances, indices = model.kneighbors(caract_film)

    if not df_test_encoded[(df_test_encoded['tconst'] == tconst) & (df_test_encoded['notes'] >= 0.7)].empty:
        selection = bons_films.iloc[indices[0]].copy()
        selection['distance_knn'] = distances[0]
    else:
        selection = bons_films.iloc[indices[0]].copy()
        selection = pd.concat([df_test_encoded[df_test_encoded['tconst'] == tconst], selection.iloc[:-1]], axis=0)
        selection['distance_knn'] = distances[0]

    # ----------------------------------------------------------
    # TF-IDF avec lemmatisation
    # ----------------------------------------------------------

    colonnes_poids = {
        'genres': 1,
        'overview_lem': 1,
        'nconst': 1
    }

    # On crée une colonne 'texte' qui combine les valeurs pondérées des colonnes spécifiées
    selection['texte'] = selection.apply(lambda row: 
        ' '.join([
            (str(row[col]) + ' ') * colonnes_poids.get(col, 1)  # Répète la valeur de la colonne selon son poids
            for col in colonnes_poids.keys()
        ]),
        axis=1
    )

    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
    tfidf_matrix = vectorizer.fit_transform(selection['texte'])

    model_tfidf = NearestNeighbors(n_neighbors=1000, metric='cosine')
    model_tfidf.fit(tfidf_matrix)

    distances_tfidf, indices_tfidf = model_tfidf.kneighbors(tfidf_matrix[0])
    selection['distance_tfidf'] = distances_tfidf[0]

    # ----------------------------------------------------------
    # Moyenne pondérée des distances
    # ----------------------------------------------------------

    poids_knn = 1
    poids_tfidf = 100

    selection['distance_ponderee'] = (
        poids_knn * selection['distance_knn']) + (poids_tfidf * selection['distance_tfidf']
    ) 

    # Tri final par la distance pondérée
    selection = selection.sort_values(by='distance_ponderee')

    # ----------------------------------------------------------
    # Résultat final
    # ----------------------------------------------------------

    return selection[['title', 'tconst', 'distance_knn', 'distance_tfidf', 'distance_ponderee', 'overview_lem', 'texte']].head(10)


In [248]:
selection2 = recommandation6('tt0104431')
selection2

,title,tconst,distance_knn,distance_tfidf,distance_ponderee,overview_lem,texte
58870,Home Alone 2: Lost in New York,tt0104431,1.732189,0.000000,1.732189,NaN,"['Comedy', 'Family', 'Adventure'] nan 333"
60538,Mrs. Doubtfire,tt0107614,1.733963,0.585894,60.323389,love irresponsible dad daniel hillard estrang...,"['Comedy', 'Drama', 'Family'] love irrespons..."
56431,Home Alone,tt0099785,2.237958,0.599287,62.166670,eight year old kevin mccallister make situati...,"['Comedy', 'Family'] eight year old kevin mc..."
106890,Harry Potter and the Chamber of Secrets,tt0295297,2.238258,0.601103,62.348586,car fly tree fight back mysterious house elf ...,"['Adventure', 'Fantasy'] car fly tree fight ..."
94835,Harry Potter and the Sorcerer's Stone,tt0241527,2.451199,0.601103,62.561527,harry potter live stair aunt uncle house whol...,"['Adventure', 'Fantasy'] harry potter live s..."
95186,The Man Who Wasn't There,tt0243133,2.837149,0.601103,62.947478,tale murder crime punishment set summer ed cr...,"['Crime', 'Drama'] tale murder crime punishm..."
141467,The Tragedy of Macbeth,tt10095582,3.742161,0.603097,64.051867,macbeth thane glamis receive prophecy trio wi...,"['Drama', 'War'] macbeth thane glamis receiv..."
75774,The Virgin Suicides,tt0159097,3.743455,0.603097,64.053160,group male friend become obsess five mysterio...,"['Drama', 'Romance'] group male friend becom..."
113551,Lost in Translation,tt0335266,3.933236,0.603097,64.242942,two lose soul visit tokyo young neglect wife ...,"['Drama', 'Romance', 'Comedy'] two lose soul..."
43580,The Deer Hunter,tt0077416,5.386922,0.603097,65.696628,group work class friend decide enlist army vi...,"['Drama', 'War'] group work class friend dec..."


In [233]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import spacy

# Charger le modèle Spacy pour la lemmatisation
nlp = spacy.load("en_core_web_sm")

def recommandation6(tconst):
    # ----------------------------------------------------------
    # Préparation des données
    # ----------------------------------------------------------
    index = df_test.index
    df_test_num = df_test.select_dtypes('number')
    df_test_cat = df_test.select_dtypes(['object', 'category', 'string', 'bool'])

    # Normalisation des colonnes numériques
    SN = MinMaxScaler()
    df_test_num_SN = pd.DataFrame(SN.fit_transform(df_test_num), columns=df_test_num.columns, index=index)

    # Encodage uniquement de la colonne 'nconst'
    df_test_cat_encoded = df_test_cat.copy()
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df_test_cat_encoded['nconst'] = le.fit_transform(df_test_cat_encoded['nconst'].fillna('inconnu'))

    # On assemble le df numérique et le df texte
    df_test_encoded = pd.concat([df_test_num_SN, df_test_cat_encoded], axis=1)

    #On sépare notre df en deux groupes, en fonction de la note
    bons_films = df_test_encoded[df_test_encoded['notes'] >= 0.7]

    # ----------------------------------------------------------
    # KNN sur les caractéristiques numériques
    # ----------------------------------------------------------
    colonnes_a_exclure = ['tconst', 'title', 'tmdb_popularity', 'title_ratings_numVotes', 'imdb_id', 'genres', 'overview', 'tmdb_US',
          'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
          'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
          'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
          'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
          'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
          'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA']
    caracteristiques = df_test_encoded.columns.drop(colonnes_a_exclure, errors='ignore')

    model = NearestNeighbors(n_neighbors=1000, metric='euclidean') # Il faudra tester d'autres combinaisons
    model.fit(bons_films[caracteristiques])

    caract_film = df_test_encoded[df_test_encoded['tconst'] == tconst][caracteristiques]
    distances, indices = model.kneighbors(caract_film)

    selection = bons_films.iloc[indices[0]].copy()
    selection['distance_knn'] = distances[0]

    # ----------------------------------------------------------
    # TF-IDF avec lemmatisation
    # ----------------------------------------------------------
    def lemmatize_text(text):
        doc = nlp(str(text))
        return " ".join([token.lemma_ for token in doc if token.is_alpha and token.text not in ENGLISH_STOP_WORDS])

    selection['overview_lem'] = selection['overview'].apply(lemmatize_text)

    colonnes_poids = {
        'genres': 1,
        'overview_lem': 1,
        'nconst': 1
    }

    # On crée une colonne 'texte' qui combine les valeurs pondérées des colonnes spécifiées
    selection['texte'] = selection.apply(lambda row: 
        ' '.join([
            (str(row[col]) + ' ') * colonnes_poids.get(col, 1)  # Répète la valeur de la colonne selon son poids
            for col in colonnes_poids.keys()
        ]),
        axis=1
    )

    vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
    tfidf_matrix = vectorizer.fit_transform(selection['texte'])

    model_tfidf = NearestNeighbors(n_neighbors=1000, metric='cosine')
    model_tfidf.fit(tfidf_matrix)

    distances_tfidf, indices_tfidf = model_tfidf.kneighbors(tfidf_matrix[0])

    selection['distance_tfidf'] = distances_tfidf[0]

    # ----------------------------------------------------------
    # Moyenne pondérée des distances
    # ----------------------------------------------------------
    poids_knn = 1
    poids_tfidf = 100

    selection['distance_ponderee'] = (
        poids_knn * selection['distance_knn']) * (poids_tfidf * selection['distance_tfidf']
    ) 

    # Tri final par la distance pondérée
    selection = selection.sort_values(by='distance_ponderee')

    # ----------------------------------------------------------
    # Résultat final
    # ----------------------------------------------------------
    return selection[['title', 'tconst', 'distance_knn', 'distance_tfidf', 'distance_ponderee', 'overview_lem', 'texte']].head(10)


In [265]:
df_ml[df_ml['tconst'] == 'tt28324464']

,tconst,startYear,runtimeMinutes,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,...,tmdb_FI,tmdb_NO,tmdb_IR,tmdb_XG,tmdb_EG,tmdb_NG,tmdb_ZA,rating,nconst,notes


In [ ]:
df_test

In [281]:
def recommandation(tconst):
    
    # ----------------------------------------------------------
    # Préparation des données
    # ----------------------------------------------------------

    index = df_test.index
    df_test_num = df_test.select_dtypes('number')
    df_test_cat = df_test.select_dtypes(['object', 'category', 'string', 'bool'])

    # Normalisation des colonnes numériques
    SN = MinMaxScaler()
    df_test_num_SN = pd.DataFrame(SN.fit_transform(df_test_num), columns=df_test_num.columns, index=index)

    # Encodage uniquement de la colonne 'nconst'
    df_test_cat_encoded = df_test_cat.copy()
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df_test_cat_encoded['nconst'] = le.fit_transform(df_test_cat_encoded['nconst'].fillna('inconnu'))

    # On assemble le df numérique et le df texte
    df_test_encoded = pd.concat([df_test_num_SN, df_test_cat_encoded], axis=1)

    #On sépare notre df en deux groupes, en fonction de la note
    bons_films = df_test_encoded[df_test_encoded['notes'] >= 0.7]

    # ----------------------------------------------------------
    # KNN sur les caractéristiques numériques
    # ----------------------------------------------------------

    colonnes_a_exclure = ['tconst', 'title', 'tmdb_popularity', 'title_ratings_numVotes', 'imdb_id', 'genres', 'overview', 'overview_lem']
    caracteristiques = df_test_encoded.columns.drop(colonnes_a_exclure, errors='ignore')

    model = NearestNeighbors(n_neighbors=1000, metric='euclidean') # Il faudra tester d'autres combinaisons
    model.fit(bons_films[caracteristiques])

    caract_film = df_test_encoded[df_test_encoded['tconst'] == tconst][caracteristiques]
    distances, indices = model.kneighbors(caract_film)

    if not df_test_encoded[(df_test_encoded['tconst'] == tconst) & (df_test_encoded['notes'] >= 0.7)].empty:
        selection = bons_films.iloc[indices[0]].copy()
        selection['distance_knn'] = distances[0]
    else:
        selection = bons_films.iloc[indices[0]].copy()
        selection = pd.concat([df_test_encoded[df_test_encoded['tconst'] == tconst], selection.iloc[:-1]], axis=0)
        selection['distance_knn'] = distances[0]

    # ----------------------------------------------------------
    # TF-IDF avec lemmatisation
    # ----------------------------------------------------------

    colonnes_poids = {
        'genres': 1,
        'overview_lem': 1,
        'nconst': 1
    }

    # On crée une colonne 'texte' qui combine les valeurs pondérées des colonnes spécifiées
    selection['texte'] = selection.apply(lambda row: 
        ' '.join([
            (str(row[col]) + ' ') * colonnes_poids.get(col, 1)  # Répète la valeur de la colonne selon son poids
            for col in colonnes_poids.keys()
        ]),
        axis=1
    )

    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
    tfidf_matrix = vectorizer.fit_transform(selection['texte'])

    model_tfidf = NearestNeighbors(n_neighbors=1000, metric='cosine')
    model_tfidf.fit(tfidf_matrix)

    distances_tfidf, indices_tfidf = model_tfidf.kneighbors(tfidf_matrix[0])
    selection['distance_tfidf'] = distances_tfidf[0]

    # ----------------------------------------------------------
    # Moyenne pondérée des distances
    # ----------------------------------------------------------

    poids_knn = 1
    poids_tfidf = 100

    selection['distance_ponderee'] = (
        poids_knn * selection['distance_knn']) + (poids_tfidf * selection['distance_tfidf']
    ) 

    # Tri final par la distance pondérée
    selection = selection.sort_values(by='distance_ponderee')

    # ----------------------------------------------------------
    # Résultat final
    # ----------------------------------------------------------
    selection_finale = pd.DataFrame(selection['tconst'][1:11]).reset_index(drop=True)

    return selection_finale

In [282]:
recommandation('tt0099785')

,tconst
0,tt0107614
1,tt0241527
2,tt0295297
3,tt0243133
4,tt10095582
5,tt0335266
6,tt0159097
7,tt0077416
8,tt0083652
9,tt0115005
